In [3]:
import random
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD

nltk.download('punkt')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

# Define intents
intents = {
    "intents": [
        {
            "tag": "greeting",
            "patterns": ["hello", "hi", "hey", "good morning"],
            "responses": ["Hello! I'm your healthcare assistant. How can I help today?"]
        },
        {
            "tag": "goodbye",
            "patterns": ["bye", "goodbye", "see you", "take care"],
            "responses": ["Goodbye! Take care of your health."]
        },
        {
            "tag": "symptoms",
            "patterns": ["i have a headache", "my stomach hurts", "i feel dizzy", "i have fever"],
            "responses": [
                "I'm sorry to hear that. How long have you had these symptoms?"
            ]
        },
        {
            "tag": "severity_followup",
            "patterns": ["1", "5", "6", "7", "10", "maybe around 6", "it's mild", "severe"],
            "responses": [
                "Got it. That sounds moderately severe. Have you taken any medication?"
            ]
        },
        {
            "tag": "medication",
            "patterns": ["i took paracetamol", "i haven't taken anything", "no, not yet"],
            "responses": [
                "You should monitor your symptoms. If they persist, consult a doctor."
            ]
        },
        {
            "tag": "appointment",
            "patterns": ["book a visit", "can i see a doctor", "schedule an appointment"],
            "responses": [
                "I can help you find a doctor. What specialty do you need?"
            ]
        },
        {
            "tag": "appointment_details",
            "patterns": ["neurologist tomorrow", "cardiologist next week", "dermatologist today"],
            "responses": [
                "Great! I’ll look into available {specialty} appointments for {time}."
            ]
        },
        {
            "tag": "noanswer",
            "patterns": [],
            "responses": ["Sorry, I didn't get that. Can you rephrase your question?"]
        }
    ]
}

# Preprocess
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = sorted(set([lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]))
classes = sorted(set(classes))

# Training
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in doc[0]]
    for w in words:
        bag.append(1 if w in pattern_words else 0)

    output_row = output_empty[:]
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)
train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

# Model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=0)

# Helpers
def clean_input(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    return [lemmatizer.lemmatize(word.lower()) for word in sentence_words]

def bag_of_words(sentence, words):
    sentence_words = clean_input(sentence)
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return np.array(bag)

def predict_tag(sentence):
    bow = bag_of_words(sentence, words)
    res = model.predict(np.array([bow]))[0]
    threshold = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > threshold]
    results.sort(key=lambda x: x[1], reverse=True)
    if results:
        return classes[results[0][0]]
    return "noanswer"

def get_response(tag, sentence=""):
    for intent in intents['intents']:
        if intent['tag'] == tag:
            if tag == "appointment_details":
                # Simple extraction
                words = sentence.lower().split()
                specialty = next((word for word in words if "ologist" in word), "specialist")
                time = next((word for word in words if word in ["today", "tomorrow", "next"]), "soon")
                return intent['responses'][0].format(specialty=specialty, time=time)
            return random.choice(intent['responses'])
    return "I'm not sure how to help with that."

# Chat with memory
def healthcare_chat():
    print("Healthcare Assistant: Hello! I'm here to help with your health questions. Type 'quit' to exit.\n")
    memory = {"last_intent": None}

    while True:
        user_input = input("You: ")
        if user_input.lower() == "quit":
            print("Healthcare Assistant: Take care of your health! Goodbye.")
            break

        intent_tag = predict_tag(user_input)

        # Follow-up logic
        if memory["last_intent"] == "symptoms" and intent_tag == "noanswer":
            intent_tag = "severity_followup"
        elif memory["last_intent"] == "appointment" and intent_tag == "noanswer":
            intent_tag = "appointment_details"

        response = get_response(intent_tag, user_input)
        print(f"Healthcare Assistant: {response}")

        memory["last_intent"] = intent_tag

        # Extra follow-ups
        if intent_tag == "symptoms":
            print("Healthcare Assistant: On a scale from 1 to 10, how severe are your symptoms?")
        elif intent_tag == "appointment":
            print("Healthcare Assistant: Do you have a preferred time or doctor?")

# Run chatbot
if __name__ == "__main__":
    healthcare_chat()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Yuvasri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Yuvasri\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Healthcare Assistant: Hello! I'm here to help with your health questions. Type 'quit' to exit.



You:  hello


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
Healthcare Assistant: Hello! I'm your healthcare assistant. How can I help today?


You:  i have a headache


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Healthcare Assistant: Goodbye! Take care of your health.


You:  bye


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Healthcare Assistant: Goodbye! Take care of your health.


You:  quit


Healthcare Assistant: Take care of your health! Goodbye.
